In [5]:
import sys
import importlib
importlib.reload(sys)
if "../" not in sys.path:
    sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
from plotly import graph_objects as go
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [9]:
length=[12, 12]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))

In [10]:
import numpy as np
a = np.tan((1/6)*np.pi) * 1/2
edge_coors_prime = np.array([
    hex_.a((1/2)*np.pi) * a,
    hex_.a(np.pi*(2/3)+(1/2)*np.pi) * a,
    hex_.a(np.pi*(4/3)+(1/2)*np.pi) * a,
])

edge_coor = np.array([[ 0.5     ,  0.      ],
                       [ 0.25    , -0.433013],
                       [ 0.75    , -0.433013]])
edge_coors_prime

K = np.zeros((2*length[0],2*length[1],2))
for l1 in range(K.shape[0]):
    for l2 in range(K.shape[1]):
        K[l1,l2] = hex_.b1*4*np.pi*l1/K.shape[0] + hex_.b2*4*np.pi*l2/K.shape[1]  


uni = np.exp(1j*np.einsum('ijk,lk->lij',K,edge_coors_prime))


# print('load P and t_list')

x = np.arange(length[0]*2)
y = np.arange(length[1]*2)

xx, yy = np.meshgrid(x,y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)
edge_coor_array = xx[:,None] * hex_.a1[None,:] * 1/2 + yy[:,None]  * hex_.a2[None,:] * 1/2

Dimer = []
total_num_samples = []

In [22]:

base_edge = edge_coor[0]

edges = hex_.edge_coor_to_lattice(hex_.ProcessPeriodic(edge_coor_array + base_edge))
colors = hex_.get_edge_color(edges)

operators = f.return_spin_corr(hi, edges, colors)

In [23]:
edges

array([[12, 24],
       [-1, -1],
       [13, 25],
       ...,
       [11, 22],
       [11, 23],
       [ 0, 23]])

In [24]:
data1 = go.Scatter(
    x = hex_.lattice_coor[:,0],
    y = hex_.lattice_coor[:,1],
    text=[f'{i}' for i in range(np.prod(hex_.l) * 2)],
    mode = 'markers+text',
    textposition="top center",
)


data2 = go.Scatter(
    x = (edge_coor_array + base_edge)[:2,0],
    y = (edge_coor_array + base_edge)[:2,1],
    mode = 'markers',
)

fig = go.Figure()
fig.add_traces(
    [data1, data2]
)
fig.layout.yaxis = {
    'scaleanchor': 'x', 'scaleratio': 1
}
fig.layout.yaxis.showgrid = False
fig.layout.xaxis.showgrid = False
fig.show()

In [15]:
ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)
ma._ws[:] = 0
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

1


In [16]:
n_jobs = 12

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
_ = sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.

number of core : 12


In [18]:
n_samples_ = 10**5
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [ ]:
import scripts.dynamics as Dynamics
from importlib import reload

reload(Dynamics)
t_list = np.arange(0,20,0.1)
d = Dynamics.new_dynamics3(op, ma)
%time P_ = d.multiprocess(samples_state, t_list, 12)
# P_ = d.dynamics(samples_state, t_list)
f.process_P2(P_)
P_list = P_

# d = Dynamics.new_dynamics(op, ma)
# %time P_ = d.multiprocess(samples_state, 300, 12) 